<a href="https://colab.research.google.com/github/kannavdhawan/commiting_Personal_baselines/blob/master/q2kannav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from keras.utils import get_file
import tarfile
data_dir = get_file('aclImdb_v1.tar.gz', 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz', cache_subdir = "datasets",hash_algorithm = "auto", extract = True, archive_format = "auto")

my_tar = tarfile.open(data_dir)
my_tar.extractall('./data/') # specify which folder to extract to
my_tar.close()

KeyboardInterrupt: ignored

In [72]:
import os
import keras 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import Word2Vec
import timeit
from keras.preprocessing.text import Tokenizer
import io
import json
import nltk
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, Dropout, Activation, Flatten,Conv1D,MaxPooling1D
from keras import regularizers


from nltk.tokenize import word_tokenize
nltk.download('punkt')

from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords=stopwords.words('english')
# import 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
def sorting_alpha(path):
    conversion_lambda = lambda text: int(text) if text.isdigit() else text.lower()
    alphanumeric_keys = lambda keys: [ conversion_lambda(char) for char in re.split('([0-9]+)', keys) ]
    sort_list=sorted(path, key=alphanumeric_keys)
    return sort_list

In [74]:
def load_data(data_path):
  X=[]
  y=[]
  for sentiment in ['pos', 'neg']:
      for file_name in sorting_alpha(os.listdir(os.path.join(data_path,sentiment))):
          if file_name.endswith('.txt'):
              with open(os.path.join(data_path,sentiment,file_name)) as f:
                  X.append(f.read())
              y.append(0 if sentiment == 'neg' else 1)
  return X,y

In [76]:
train_features,train_labels=load_data(os.path.join("/content/data",'aclImdb','train'))
test_features,test_labels=load_data(os.path.join("/content/data",'aclImdb','test'))

In [77]:

"""
Type train features:  <class 'list'>
Length of train features:  25000
Type test features:  <class 'list'>
Length test features:  25000
"""

print("Type train features: ",type(train_features))
print("Length of train features: ",len(train_features))
print("Type test features: ",type(test_features))
print("Length test features: ",len(test_features))


Type train features:  <class 'list'>
Length of train features:  25000
Type test features:  <class 'list'>
Length test features:  25000


In [78]:
# converting into DataFrames
train=pd.concat([pd.DataFrame(train_features,columns=['Text']),pd.DataFrame(train_labels,columns=['Label'])],axis=1)
test=pd.concat([pd.DataFrame(test_features,columns=['Text']),pd.DataFrame(test_labels,columns=['Label'])],axis=1)
assert train.shape==test.shape

In [79]:
print(train.head(2))
print(train.tail(2))
print(test.head(2))
print(test.tail(2))

                                                Text  Label
0  Bromwell High is a cartoon comedy. It ran at t...      1
1  If you like adult comedy cartoons, like South ...      1
                                                    Text  Label
24998  This film has the kernel of a really good stor...      0
24999  I went to the movie as a Sneak Preview in Aust...      0
                                                Text  Label
0  I went and saw this movie last night after bei...      1
1  My boyfriend and I went to watch The Guardian....      1
                                                    Text  Label
24998  Oh, dear! This has to be one of the worst film...      0
24999  This movie was sooo bad. It wasn't even funny ...      0


In [80]:
# # Finding all punctuations in train set..
# set_punc=set()
# for reviews in train['Text']:
#   for chr in reviews:
#     set_punc=set_punc.union(set(re.findall(r"[^a-zA-Z0-9\s]",chr)))

# print("Punctuations in our dataset: ",set_punc)

# relevant punctuations updated from above list
punctuations="[\s.;:!\'?,\"()\[\]/_ÉºèíÁ{()ïá`&₤%äÜô“êðßÈ³¾âø;Ø\®»!-=ñÀöã?!#$%&'()*+,-./:;<=>?@[\]^_`{|}~å°@0¨ë:¢û*$´ùóüçúî~½<’æ‘§}£.«ÕÊì¤ÃÄ·,éý|-Åō#ò¿–à><]"

In [99]:
def punctuation_remover(series,punctuations):
    series_list=[re.sub(punctuations," ",row) for row in series]
    return series_list

def punctuation_remover_basic(list_text):
    list_text=[re.sub(r'[^a-zA-Z0-9]+'," ",review) for review in list_text]
    list_text=[re.sub(r'\s+'," ",review) for review in list_text]
    list_text=[re.sub(r'(<br\s*/><br\s*/>)|(\-)|(\/)'," ",review) for review in list_text]
    return list_text

def lower_text(list_text):
    list_text=[review.lower() for review in list_text]
    return list_text

def stopwords_removal(list_text,stopword_list):
    list_text_nsw=[]
    for review in list_text:
        list_text_nsw.append(' '.join([word for word in review.split() if word not in stopword_list]))
    return list_text_nsw

def tokenize_word_sentences(list_text):       
    tokenized_list = []
    for review in list_text:
        tokenized_list.append(word_tokenize(review))  # list of list 
          
    return tokenized_list      

def w2v_embedding(formatted_dataset):
    print("Training...")
    start=timeit.default_timer()
    w2v=Word2Vec(sentences=formatted_dataset,min_count=4, size=350,window=2,workers=4,iter=5) #sample=e-5, alpha=0.01,min_alpha=0.0001
    stop=timeit.default_timer()
    print("Time taken: ",stop-start)
    w2v.save("word2vec.model")
    return w2v

def most_sim(model,word,n):
    print("\n\n---------------",word,": Most similar words---------------")
    try:
        alltups=[]
        
        for i in range(n):
            tup=model.most_similar(positive=[word], topn=n)[i]
            alltups.append(tup)
        for k, v in dict(alltups).items():
            print (k, '-->', v)
    except:
        print("word : ",word," not in vocab")

def fit_on_text(data):
    
   
    length_list=[len(seq) for seq in data]
    avg=sum(length_list)/len(length_list)
    max_length= int(avg+(max(length_list)-avg))# average number of words in each sentence.   
    max_length=500 
    print("--------------------------------------------------------------------------------------------------",max_length)
    
    token=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
 
    list_of_strings_full_data=[' '.join(seq[:max_length]) for seq in data]
    token.fit_on_texts(list_of_strings_full_data)
    
    tokenizer_json = token.to_json()
    with io.open(os.path.join('tokenizer.json'), 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))


    return max_length,token

def texts_to_sequences(token,max_length,X_train,X_test):
    tr=[' '.join(seq[:max_length]) for seq in X_train]  #['This product is very good','']
    # vl=[' '.join(seq[:max_length]) for seq in X_val]    #['This product is very good','']
    tst=[' '.join(seq[:max_length]) for seq in X_test]  #['This product is very good','']
    print(tr[0:1])
    X_train = token.texts_to_sequences(tr)
    # X_val = token.texts_to_sequences(vl)
    X_test = token.texts_to_sequences(tst)

    X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
    # X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')
    X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')
    print("shape test 1:",X_train.shape)
    print(X_train[0])
    return X_train,X_test

def embedding_matrix(token):

    w2v_embeddings = Word2Vec.load('word2vec.model')
   
    e_dim=w2v_embeddings.vector_size #350

    print("vector size embedding",e_dim)


    v_size=len(token.word_index)+1 

    print("vocabulary_size: ",v_size)
    
    # making the embedding matrtix and feeding with array from word2vec embeddings.

    embed_matrix=np.random.randn(v_size,e_dim) #114556*350
    for word,index in token.word_index.items():
        if word in w2v_embeddings.wv.vocab:
            embed_matrix[index]=w2v_embeddings[word]#feeding the embedding matrix with array from word2vec embeddings
        else:
            embed_matrix[index]=np.random.randn(1,e_dim) # if word from word index is not there in word2vec embeddings, input randomly.
    


    return e_dim,v_size,embed_matrix


def to_df(X_train,X_test,y_train,y_test):
    

    X_train=pd.DataFrame(X_train)
    y_train=pd.DataFrame(y_train)
    
    X_test=pd.DataFrame(X_test)
    y_test=pd.DataFrame(y_test)
    
    return X_train,X_test,y_train,y_test


def model(X_train,X_test,max_length,e_dim,v_size,e_mat,y_train,y_test,act_func,l2_norm_f,l2_norm,dropout_f,dropout):

    # Embedding layer
    clf=Sequential()
    e_layer=Embedding(input_dim=v_size,output_dim=e_dim,weights=[e_mat], input_length=max_length,
                            trainable=False)                              #(114556,350,[114556*350],24)==>(None,24,350) i.e. (input_length,output_dim)
    clf.add(e_layer)
    #Flatten
    clf.add(Flatten()) #flatten
    #Dense
    if l2_norm_f==True:
        clf.add(Dense(128,activation=act_func,kernel_regularizer=regularizers.l2(l2_norm)))
    else:
        clf.add(Dense(128,activation=act_func))
    #dropout
    if dropout_f==True:
        clf.add(Dropout(dropout))
    else:
        pass
    clf.add(Dense(2,activation='softmax'))# final layer
    clf.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    # print(clf.summary())
    
    history=clf.fit(X_train, y_train,batch_size=1024,epochs=15)

    test_score,test_acc = clf.evaluate(X_test,y_test,batch_size=1024)
    print("Test Accuracy : ", test_acc*100)

    return test_acc*100,history

def model1(vocab_len, model_dim, weights_matrix, len_review, X_train, X_test, y_train, y_test):

  model = Sequential()

  model.add(Embedding(input_dim = vocab_len, output_dim = model_dim, weights = [weights_matrix], input_length = len_review,trainable=True))

  model.add(Conv1D(32, 3, padding='same', activation='relu'))

  model.add(MaxPooling1D())

  model.add(Flatten())

  model.add(Dense(128, activation = 'relu'))

  model.add(Dropout(rate = 0.3))

  model.add(Dense(2,activation ='softmax'))

  model.compile(loss ='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  
  model.fit(X_train, y_train, batch_size = 100, epochs = 5)
  
  acc_score = model.evaluate(X_test,y_test)

  print("Test Accuracy is {}% ".format(acc_score[1] * 100))
  
  model.save("m.model")

In [100]:
# removing punctuations
train_text=punctuation_remover(train['Text'],punctuations) # list
train_text=punctuation_remover_basic(train_text)           # list

# Removing punctuations additionally 

test_text=punctuation_remover(test['Text'],punctuations)
test_text=punctuation_remover_basic(test_text)           # list

# lowercasing

train_text=lower_text(train_text)
test_text=lower_text(test_text)

# Stopword removal
stopword_list_1=[ 'we', 'our', 'ours', 'ourselves', 'you', "you're",'i', 'me', 'my', 'myself', "you've", "you'll", "you'd", 'your',
                     'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
                      'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves']
stopword_list_2=["a", "about", 'other', 'some', 'such',  'own', 'same', 'so', 'than','s',
                          'too', 'to', 'from', 'in', 'out', 'on', 'off', 'over', 'under', 
                        'then', 'once', 'here', 'there', 'all', 'any', 'both', 'each',
                          't', 'can', 'will', 'just', 'don', 'now', 'd','s']

stopword_list=list(set(stopword_list_1).union(set(stopword_list_2)))

train_text=stopwords_removal(train_text,stopword_list)
test_text=stopwords_removal(test_text,stopword_list)

In [101]:
# train_text[0]

In [102]:
train_text_tokenized=tokenize_word_sentences(train_text)
test_text_tokenized=tokenize_word_sentences(test_text)

In [103]:
w2v=w2v_embedding(train_text_tokenized)

Training...
Time taken:  62.813864585000374


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [104]:
w2v=Word2Vec.load('word2vec.model') 

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [105]:
most_sim(w2v,"good",20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).




--------------- good : Most similar words---------------
decent --> 0.7245098352432251
fine --> 0.6855379343032837
great --> 0.6772593259811401
bad --> 0.6638052463531494
cool --> 0.649214506149292
nice --> 0.6345492601394653
solid --> 0.6197190284729004
excellent --> 0.6182861328125
terrific --> 0.6178426742553711
fantastic --> 0.6020016670227051
ok --> 0.6013531684875488
impressive --> 0.5881668925285339
interesting --> 0.5815615057945251
funny --> 0.5760958790779114
awesome --> 0.5756717920303345
lousy --> 0.5749622583389282
fair --> 0.5739405751228333
mediocre --> 0.5730897784233093
alright --> 0.5657459497451782
scary --> 0.5645594000816345


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [106]:
max_length,token=fit_on_text(train_text_tokenized)

-------------------------------------------------------------------------------------------------- 500


In [107]:
X_train,X_test=texts_to_sequences(token,max_length,train_text_tokenized,test_text_tokenized)

['bromwell high is cartoon comedy ran at the time as programs school life as teachers years the teaching profession lead believe that bromwell high satire is much closer reality is teachers the scramble survive financially the insightful students who see right through pathetic teachers pomp the pettiness of the whole situation remind of the schools knew and students when saw the episode which student repeatedly tried burn down the school immediately recalled at high classic line inspector m sack one of teachers student welcome bromwell high expect that many adults of age think that bromwell high is far fetched what pity that isn']
shape test 1: (25000, 500)
[20466   255     4   983   159  2086    20     1    34     9  5751   324
    74     9  4990   109     1  4804  5752   415   210     7 20466   255
  1883     4    46  2323   566     4  4990     1 23538  1903  9810     1
  5816  1412    23    42   158   102  1139  4990 19364     1 45538     3
     1   175   833  3014     3     1  5564

In [108]:
e_dim,v_size,embed_matrix=embedding_matrix(token)

vector size embedding 350
vocabulary_size:  72096


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [109]:
print(X_train.shape)
print(X_test.shape)

(25000, 500)
(25000, 500)


In [110]:
  # Converting into categorical data 
     
y_train=np_utils.to_categorical(train['Label'])
y_test=np_utils.to_categorical(test['Label'])

In [111]:
X_train,X_test,y_train,y_test=to_df(X_train,X_test,y_train,y_test)

In [112]:
X_train.shape

(25000, 500)

In [113]:
y_test.shape

(25000, 2)

In [114]:
y_train

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
24995,1.0,0.0
24996,1.0,0.0
24997,1.0,0.0
24998,1.0,0.0


In [115]:
acc1,history1=model(X_train,X_test,max_length,e_dim,v_size,embed_matrix,y_train,y_test,'relu',False,0,False,0) # dropout=False | l2 False

Epoch 1/15
25000/25000 [==============================] - 3s 113us/step - loss: 6.9322 - accuracy: 0.5130
Epoch 2/15
25000/25000 [==============================] - 3s 102us/step - loss: 7.1232 - accuracy: 0.5312
Epoch 3/15
25000/25000 [==============================] - 3s 101us/step - loss: 7.0905 - accuracy: 0.5352
Epoch 4/15
25000/25000 [==============================] - 3s 102us/step - loss: 7.2293 - accuracy: 0.5293
Epoch 5/15
25000/25000 [==============================] - 3s 101us/step - loss: 7.1656 - accuracy: 0.5331
Epoch 6/15
25000/25000 [==============================] - 3s 101us/step - loss: 7.1271 - accuracy: 0.5334
Epoch 7/15
25000/25000 [==============================] - 3s 102us/step - loss: 7.0331 - accuracy: 0.5388
Epoch 8/15
25000/25000 [==============================] - 3s 101us/step - loss: 7.0092 - accuracy: 0.5404
Epoch 9/15
25000/25000 [==============================] - 3s 101us/step - loss: 6.9873 - accuracy: 0.5426
Epoch 10/15
25000/25000 [=====================

In [116]:
model1(v_size, e_dim, embed_matrix, max_length, X_train, X_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
25000/25000 [==============================] - 12s 495us/step - loss: 0.6859 - accuracy: 0.6225
Epoch 2/5
25000/25000 [==============================] - 12s 490us/step - loss: 0.2902 - accuracy: 0.8853
Epoch 3/5
25000/25000 [==============================] - 12s 487us/step - loss: 0.1311 - accuracy: 0.9544
Epoch 4/5
25000/25000 [==============================] - 12s 491us/step - loss: 0.0450 - accuracy: 0.9870
Epoch 5/5
25000/25000 [==============================] - 4s 164us/step
Test Accuracy is 86.10399961471558% 


- Remove punctuations
- Remove stopwords
- Lemmatization
- Tokenize
- embedding word2vec 
- fit on text keras
- Text  to seq keras 
- pad seq keras 
- embed mat manually 
- Model keras conv





- To check for accuracy
  - stopword list 